In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st
from babel.numbers import format_currency

In [2]:
sns.set(style='dark')

In [19]:
df = pd.read_csv("D:/Dataanalyst/python/baru/latihan/all_data.csv")

In [20]:
def create_daily_orders_df(df):
    daily_orders_df = df.resample(rule='D', on='order_date').agg({
        "order_id": "nunique",
        "total_price": "sum"
    })
    daily_orders_df = daily_orders_df.reset_index()
    daily_orders_df.rename(columns={
        "order_id": "order_count",
        "total_price": "revenue"
    }, inplace=True)
    
    return daily_orders_df

In [21]:
def create_sum_order_items_df(df):
    sum_order_items_df = df.groupby("product_name").quantity_x.sum().sort_values(ascending=False).reset_index()
    return sum_order_items_df

In [22]:
def create_bygender_df(df):
    bygender_df = df.groupby(by="gender").customer_id.nunique().reset_index()
    bygender_df.rename(columns={
        "customer_id": "customer_count"
    }, inplace=True)
    
    return bygender_df

In [23]:
def create_byage_df(df):
    byage_df = df.groupby(by="age_group").customer_id.nunique().reset_index()
    byage_df.rename(columns={
        "customer_id": "customer_count"
    }, inplace=True)
    byage_df['age_group'] = pd.Categorical(byage_df['age_group'], ["Youth", "Adults", "Seniors"])
    
    return byage_df

In [25]:
def create_bystate_df(df):
    bystate_df = df.groupby(by="state").customer_id.nunique().reset_index()
    bystate_df.rename(columns={
        "customer_id": "customer_count"
    }, inplace=True)
    
    return bystate_df

In [26]:
def create_rfm_df(df):
    rfm_df = df.groupby(by="customer_id", as_index=False).agg({
        "order_date": "max", #mengambil tanggal order terakhir
        "order_id": "nunique",
        "total_price": "sum"
    })
    rfm_df.columns = ["customer_id", "max_order_timestamp", "frequency", "monetary"]
    
    rfm_df["max_order_timestamp"] = rfm_df["max_order_timestamp"].dt.date
    recent_date = df["order_date"].dt.date.max()
    rfm_df["recency"] = rfm_df["max_order_timestamp"].apply(lambda x: (recent_date - x).days)
    rfm_df.drop("max_order_timestamp", axis=1, inplace=True)
    
    return rfm_df

In [27]:
print(df)

      sales_id  order_id  product_id  price_per_unit  quantity_x  total_price  \
0            0         1         218             106           2          212   
1            1         1         481             118           1          118   
2            2         1           2              96           3          288   
3            3         1        1002             106           2          212   
4            4         1         691             113           3          339   
...        ...       ...         ...             ...         ...          ...   
4995      4995       998         321             109           2          218   
4996      4996       998         251              95           3          285   
4997      4997       999         872             113           1          113   
4998      4998       999         998             106           2          212   
4999      4999       999        1105             115           1          115   

     product_type     produ

In [36]:
df = df.dropna()

In [ ]:
# numerik jadi 0
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[numeric_cols] = df[numeric_cols].fillna(0)

In [ ]:
# kategorik jadi unknown
categorical_cols = df.select_dtypes(include=['object']).columns
df[categorical_cols] = df[categorical_cols].fillna('Unknown')

In [39]:
# Display the cleaned data
print(df.head())

   sales_id  order_id  product_id  price_per_unit  quantity_x  total_price  \
0         0         1         218             106           2          212   
1         1         1         481             118           1          118   
2         2         1           2              96           3          288   
3         3         1        1002             106           2          212   
4         4         1         691             113           3          339   

  product_type  product_name size  colour  ...  customer_name  \
0        Shirt      Chambray    L  orange  ...       fulan 64   
1       Jacket        Puffer    S  indigo  ...       fulan 64   
2        Shirt  Oxford Cloth    M     red  ...       fulan 64   
3     Trousers          Wool    M    blue  ...       fulan 64   
4       Jacket         Parka    S  indigo  ...       fulan 64   

              gender age               home_address  zip_code            city  \
0  Prefer not to say  75  4927 Alice MeadowApt. 960      77

In [40]:
print(df)

      sales_id  order_id  product_id  price_per_unit  quantity_x  total_price  \
0            0         1         218             106           2          212   
1            1         1         481             118           1          118   
2            2         1           2              96           3          288   
3            3         1        1002             106           2          212   
4            4         1         691             113           3          339   
...        ...       ...         ...             ...         ...          ...   
4995      4995       998         321             109           2          218   
4996      4996       998         251              95           3          285   
4997      4997       999         872             113           1          113   
4998      4998       999         998             106           2          212   
4999      4999       999        1105             115           1          115   

     product_type     produ

In [41]:
# Save the cleaned DataFrame to the specified path
df.to_csv(r'D:\Dataanalyst\python\baru\latihan\project\clean_data.csv', index=False)
